In [1]:
%use kandy
import generator.Generator
import generator.Point
import generator.Sector
import kotlin.random.Random

In [ ]:
import generator.Sector
import org.apache.commons.math3.distribution.PoissonDistribution


val A = 10
val distance = 5
val (p1, p2) = Generator.nextPariInSquare(distance, A)

val s1 = Sector(p1, Random.nextDouble(0.0, 2 * PI), PI / 2)
val s2 = Sector(p2, Random.nextDouble(0.0, 2 * PI), PI / 2)

val fovDist = distance

val s1de = s1.getDirectionEndPoint(fovDist)
val s1le = s1.getLineEndsInSquare(fovDist)
val s2de = s2.getDirectionEndPoint(fovDist)
val s2le = s2.getLineEndsInSquare(fovDist)


val obstacles = HashSet<Point>()
val a = 0.1
val radius = 0.1
repeat(PoissonDistribution(a * A * A).sample()) {
    obstacles.add(Generator.nextPointInSquare(A))
}

var block = false
check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
    for (p in obstacles) {
        if (i.isInCircle(p, radius)) {
            block = true
            break@check
        }
    }
}
println(s1)
println(s2)
println(s1.contains(p2))
println(s2.contains(p1))

In [ ]:
import org.jetbrains.letsPlot.geom.extras.arrow

val minX = min(listOf(s1de, s2de, s1le.first, s1le.second, s2le.first, s2le.second).map { it.x }.min(), 0.0)
val minY = min(listOf(s1de, s2de, s1le.first, s1le.second, s2le.first, s2le.second).map { it.y }.min(), 0.0)

val maxX = max(listOf(s1de, s2de, s1le.first, s1le.second, s2le.first, s2le.second).map { it.x }.max(), A.toDouble())
val maxY = max(listOf(s1de, s2de, s1le.first, s1le.second, s2le.first, s2le.second).map { it.y }.max(), A.toDouble())

val psize = 25
plot {

    points {
        x(obstacles.map { it.x }) {
            scale = continuous(minX, maxX)
        }
        y(obstacles.map { it.y }) {
            scale = continuous(minY, maxY)
        }
        size = psize.toDouble() * radius
    }

    points {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(minX, maxX)
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(minY, maxY)
        }
        color = Color.RED
    }

    line {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(minX, maxX)
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(minY, maxY)
        }
        type = LineType.DASHED
        color = Color.RED
    }
    
    for (p in listOf(
        Pair(s1, s1de),
        Pair(s2, s2de)
    )) {
        line {
            x(listOf(p.first.pivot, p.second).map { it.x }) {
                scale = continuous(minX, maxX)
            }
            y(listOf(p.first.pivot, p.second).map { it.y }) {
                scale = continuous(minY, maxY)
            }
            type = LineType.DASHED
            color = Color.GREEN
            alpha = .6
        }
    }

    for (p in listOf(
        Pair(s1, s1le.first),
        Pair(s1, s1le.second),
        Pair(s2, s2le.first),
        Pair(s2, s2le.second),
    )) {
        line {
            x(listOf(p.first.pivot, p.second).map { it.x }) {
                scale = continuous(minX, maxX)
            }
            y(listOf(p.first.pivot, p.second).map { it.y }) {
                scale = continuous(minY, maxY)
            }
            type = LineType.DASHED
            color = Color.YELLOW
            alpha = .3
        }
    }

    layout {
        size = Pair(A * psize * 2, A * psize * 2)
        style {
            axis {
//                blankAxes()
            }
        }
    }
}

In [29]:
import org.apache.commons.math3.distribution.PoissonDistribution
import java.text.DecimalFormat

val repeats = 1000

val A = 10 // square side

val hs = listOf<Number>(0.1, 1) // point distance
val `as` = listOf<Number>(0.01, 0.05, 0.1, 0.3) // obstacle density
val rs = listOf<Number>(0.03, 0.1, 1) // obstacle size

val fov = PI / 2

println("a\th\tr\tImitation\tModel")
for (a in `as`) {
    val poissonDistribution = PoissonDistribution(a.toDouble() * A * A)
    for (h in hs) {
        for (r in rs) {
            var blocks = 0
            repeat(repeats) {
                val (p1, p2) = Generator.nextPariInSquare(h, A)
                val s1 = Sector(p1, Random.nextDouble(0.0, 2 * PI), fov)
                val s2 = Sector(p2, Random.nextDouble(0.0, 2 * PI), fov)
                val obstacles = HashSet<Point>()
                repeat(poissonDistribution.sample()) {
                    obstacles.add(Generator.nextPointInSquare(A))
                }
                var block = !(s1.contains(p2) and s2.contains(p1))
                if (!block) {
                    check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
                        for (p in obstacles) {
                            if (i.isInCircle(p, r)) {
                                block = true
                                break@check
                            }
                        }
                    }
                }
                if (block) {
                    ++blocks
                }
            }
            println("$a\t$h\t$r\t${DecimalFormat("#0.00#").format(blocks / repeats.toDouble())}\t${DecimalFormat("#0.00#").format(mathModel(fov, a, h, r))}")
        }
    }
}

a	h	r	Imitation	Model
0.01	0.1	0.03	0.936	0.938
0.01	0.1	0.1	0.943	0.938
0.01	0.1	1	0.945	0.938
0.01	1	0.03	0.944	0.938
0.01	1	0.1	0.941	0.938
0.01	1	1	0.948	0.939
0.05	0.1	0.03	0.943	0.938
0.05	0.1	0.1	0.948	0.938
0.05	0.1	1	0.944	0.938
0.05	1	0.03	0.944	0.938
0.05	1	0.1	0.949	0.938
0.05	1	1	0.955	0.943
0.1	0.1	0.03	0.955	0.938
0.1	0.1	0.1	0.94	0.938
0.1	0.1	1	0.965	0.939
0.1	1	0.03	0.952	0.938
0.1	1	0.1	0.941	0.939
0.1	1	1	0.96	0.949
0.3	0.1	0.03	0.946	0.938
0.3	0.1	0.1	0.93	0.938
0.3	0.1	1	0.975	0.941
0.3	1	0.03	0.941	0.939
0.3	1	0.1	0.958	0.941
0.3	1	1	0.984	0.966


In [ ]:
import org.apache.commons.math3.distribution.PoissonDistribution

val A = 10 // square side

val h = 1 // point distance
val a = 0.1 // obstacle density
val r = 0.3 // obstacle size

val poissonDistribution = PoissonDistribution(a * A * A)

val (p1, p2) = Generator.nextPariInSquare(h, A)
val obstacles = HashSet<Point>()
repeat(poissonDistribution.sample()) {
    obstacles.add(Generator.nextPointInSquare(A))
}

val psize = 25
plot {
    
    points { 
        x(obstacles.map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(obstacles.map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        size = psize.toDouble() * r
    }

    points {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        color = Color.RED
    }

    line {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        type = LineType.DASHED
        color = Color.RED
    }

    layout {
        size = Pair(A * psize, A * psize)
        style { 
            axis {
                blankAxes()
            }
        }
    }
}

In [ ]:
fun mathModel(angle: Number, lambda: Number, h: Number, r: Number): Double {
    val sector = angle.toDouble() / (2 * PI)
    return 1 - (sector.pow(2) * (exp(-lambda.toDouble() * h.toDouble() * 2 * r.toDouble())))
}

In [3]:
data class Parameters(val h: Number, val a: Number, val r: Number, val fov: Number)
data class Results(val imitation: Number, val model: Number)

In [8]:
import org.apache.commons.math3.distribution.PoissonDistribution
import java.text.DecimalFormat
import kotlin.random.Random

val repeats = 1000

val A = 10 // square side

//val hs = listOf(0.1, 1) // point distance
//val hs = listOf(1) // point distance
val hs = 1..10 // point distance
//val hs = 1..10 // point distance
val `as` = listOf(0.0001) // obstacle density
//val `as` = (1..100).map{it / 100.0} // obstacle density
val rs = listOf(0.1) // obstacle size

//val fovs = listOf(PI / 4, PI / 3, PI / 2, 2 * PI / 3, PI)
val fovs = listOf(PI / 2)
//val fovs = listOf(PI)

val result = HashMap<Number, Number>()

println("a\th\tr\tImitation\tModel")
for (f in fovs) {
    for (a in `as`) {
        val poissonDistribution = PoissonDistribution(a.toDouble() * A * A)
        for (h in hs) {
            for (r in rs) {
                var blocks = 0
                repeat(repeats) {
                    val (p1, p2) = Generator.nextPariInSquare(h, A)
                    val s1 = Sector(p1, Random.nextDouble(0.0, 2 * PI), f)
                    val s2 = Sector(p2, Random.nextDouble(0.0, 2 * PI), f)
                    val obstacles = HashSet<Point>()
                    repeat(poissonDistribution.sample()) {
                        obstacles.add(Generator.nextPointInSquare(A))
                    }
                    var block = !(s1.contains(p2) and s2.contains(p1))
                    if (!block) {
                        check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
                            for (p in obstacles) {
                                if (i.isInCircle(p, r)) {
                                    block = true
                                    break@check
                                }
                            }
                        }
                    }
                    if (block) {
                        ++blocks
                    }
                }
                result[h] = blocks / repeats.toDouble()
//                println("$a\t$h\t$r\t${DecimalFormat("#0.00#").format(blocks / repeats.toDouble())}\t${DecimalFormat("#0.00#").format(mathModel(f, a, h, r))}")
            }
        }
    }
}

plot {
    line {
        y(result.values)
        x(result.keys)
    }
    layout {
        xAxisLabel = "h"
        yAxisLabel = "P block"
    }
}

//println(result)

a	h	r	Imitation	Model


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bvHdSQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"h"
},{
"aesthetic":"y",
"name":"P block"
}],
"layers":[{
"mapping":{
"y":"y",
"x":"x"
},
"stat":"identity",
"data":{
"x":[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0],
"y":[0.944,0.944,0.948,0.957,0.943,0.948,0.95,0.956,0.943,0.942]
},
"sampling":"none",
"position":"identity",
"geom":"line"
}]
};
 var plotContainer = document.getElementById("bvHdSQ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 
 
 0.942 
 
 
 
 
 
 
 0.944 
 
 
 
 
 
 
 0.946 
 
 
 
 
 
 
 0.948 
 
 
 
 
 
 
 0.950 
 
 
 
 
 
 
 0.952 
 
 
 
 
 
 
 0.954 
 
 
 
 
 
 
 0.956 
 
 
 
 
 
 
 
 
 P block 
 
 
 
 
 h